### Index
1. 준비
2. 데이터 전처리 & EDA
3. 모델링

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### 1. 준비

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sklearn.metrics as metrics
from sklearn.metrics import f1_score, accuracy_score, recall_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neural_network import MLPClassifier 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks, NearMiss

In [7]:
train = pd.read_csv("/content/drive/MyDrive/데이크루/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/데이크루/data/test.csv")

In [8]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  301 non-null    object 
 2   sido                301 non-null    object 
 3   sgg                 301 non-null    int64  
 4   openDate            301 non-null    int64  
 5   bedCount            296 non-null    float64
 6   instkind            300 non-null    object 
 7   revenue1            293 non-null    float64
 8   salescost1          293 non-null    float64
 9   sga1                293 non-null    float64
 10  salary1             293 non-null    float64
 11  noi1                293 non-null    float64
 12  noe1                293 non-null    float64
 13  interest1           293 non-null    float64
 14  ctax1               293 non-null    float64
 15  profit1             293 non-null    float64
 16  liquidAs

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             127 non-null    int64  
 1   OC                  0 non-null      float64
 2   sido                127 non-null    object 
 3   sgg                 127 non-null    int64  
 4   openDate            126 non-null    float64
 5   bedCount            119 non-null    float64
 6   instkind            125 non-null    object 
 7   revenue1            125 non-null    float64
 8   salescost1          125 non-null    float64
 9   sga1                125 non-null    float64
 10  salary1             125 non-null    float64
 11  noi1                125 non-null    float64
 12  noe1                125 non-null    float64
 13  interest1           125 non-null    float64
 14  ctax1               125 non-null    float64
 15  profit1             125 non-null    float64
 16  liquidAs

None

#### 2. 데이터 전처리
train / test의 데이터 타입이 상이함
- (1) OC: {train: object, test: float64} > test의 Nan 값은 결과값이며, train의 ' close' 공백 제거
- (2) openDate: {train: int64, test: float64} > int
- (3-1) employee1: {train: float64, test: object} > float, ',' 제거
- (3-2) employee2: {train: float64, test: object} > float, ',' 제거

추가 전처리
- (1) employee2(2016) null값 중 다음해 employee1(2017)의 값이 존재한다면 대체했음. 2017년도 운영이 지속되었으니 인원수에 큰 변화는 없을 것으로 판단됨. 

나머지 결측치 처리
- 연속형변수: -1
- 범주형변수: unknown

In [9]:
# (1)
train['OC'] = train['OC'].apply(lambda x : x.replace(' ','')) # ' close' 공백제거

In [10]:
# (2)
test['openDate'].fillna(-1, inplace=True) # 연속형 변수 결측치 -1 처리
test['openDate'] = test['openDate'].apply(lambda x : int(x))

In [11]:
# (3)
test['employee1'] = test['employee1'].apply(lambda x: float(str(x).replace(',','')))
test['employee2'] = test['employee2'].apply(lambda x: float(str(x).replace(',','')))

In [12]:
numeric_col = train.describe().columns # 연속형 변수
categorical_col = train.describe(include='object').columns # 범주형 변수

# 변수들 범위 및 클래스
print('####################수치형변수-Train####################')
for col in numeric_col:
    print(col, ':', train[col].max(), '~', train[col].min())
print('####################수치형변수-Test####################')    
for col in numeric_col:
    print(col, ':', test[col].max(), '~', train[col].min())
print('####################범주형변수-Train####################')
for col in categorical_col:
    print(col,':', train[col].unique(),'\n')
print('####################범주형변수-Test####################')
for col in categorical_col:
    print(col,':', test[col].unique(),'\n')

####################수치형변수-Train####################
inst_id : 428 ~ 1
sgg : 178 ~ 1
openDate : 20170607 ~ 19780124
bedCount : 656.0 ~ 0.0
revenue1 : 151000000000.0 ~ 0.0
salescost1 : 98503322990.0 ~ 0.0
sga1 : 103000000000.0 ~ 0.0
salary1 : 64035593950.0 ~ 0.0
noi1 : 9144170639.0 ~ 0.0
noe1 : 8686379500.0 ~ 0.0
interest1 : 2841474870.0 ~ 0.0
ctax1 : 2808883191.0 ~ 0.0
profit1 : 7475426881.0 ~ -4696700774.0
liquidAsset1 : 42808356695.0 ~ 0.0
quickAsset1 : 41791456468.0 ~ 0.0
receivableS1 : 14092571696.0 ~ 0.0
inventoryAsset1 : 2067763564.0 ~ 0.0
nonCAsset1 : 156000000000.0 ~ 0.0
tanAsset1 : 156000000000.0 ~ 0.0
OnonCAsset1 : 10085168680.0 ~ 0.0
receivableL1 : 73742310.0 ~ 0.0
debt1 : 86697249913.0 ~ 0.0
liquidLiabilities1 : 39373879374.0 ~ 0.0
shortLoan1 : 19885197610.0 ~ 0.0
NCLiabilities1 : 55017362619.0 ~ 0.0
longLoan1 : 41124897062.0 ~ 0.0
netAsset1 : 139000000000.0 ~ -20196244362.0
surplus1 : 134000000000.0 ~ -2914969563.0
revenue2 : 137000000000.0 ~ 0.0
salescost2 : 86581590331.0 

In [13]:
# 추가 전처리(1)
train.loc[train['inst_id']==58,'employee2'] = train.loc[train['inst_id']==58,'employee1']
train.loc[train['inst_id']==96,'employee2'] = train.loc[train['inst_id']==96,'employee1']
train.loc[train['inst_id']==327,'employee2'] = train.loc[train['inst_id']==327,'employee1']
train.loc[train['inst_id']==427,'employee2'] = train.loc[train['inst_id']==427,'employee1']

test.loc[test['inst_id']==48,'employee2'] = test.loc[test['inst_id']==48,'employee1']
test.loc[test['inst_id']==246,'employee2'] = test.loc[test['inst_id']==246,'employee1']
test.loc[test['inst_id']==254,'employee2'] = test.loc[test['inst_id']==254,'employee1']
test.loc[test['inst_id']==374,'employee2'] = test.loc[test['inst_id']==374,'employee1']
test.loc[test['inst_id']==395,'employee2'] = test.loc[test['inst_id']==395,'employee1'] # 중복행
test.loc[test['inst_id']==396,'employee2'] = test.loc[test['inst_id']==396,'employee1'] # 중복행
test.loc[test['inst_id']==424,'employee2'] = test.loc[test['inst_id']==424,'employee1']

In [14]:
# 결측치 처리(연속형변수: -1, 범주형변수: 'unknown')
data = pd.concat([train, test], ignore_index=True) # Train, Test 합치기 -> 동시에 처리
data['OC'] = data['OC'].fillna('unknown_OC') # 범주형변수 OC
data['instkind'] = data['instkind'].fillna('unknown_instkind') # 범주형변수instkind
data['ownerChange'] = data['ownerChange'].fillna('unknown_ownerChange') # 범주형변수ownerChange
data[numeric_col] = data[numeric_col].fillna(-1)
print(data.shape)

(428, 58)


In [15]:
# 라벨인코딩, 종속변수(OC)에 대해서 1,0 값을 가진다.
label_encoder = LabelEncoder()
data['OC'] = label_encoder.fit_transform(data['OC'])

In [16]:
# 원핫인코딩 준비
dummy_col=categorical_col.drop('OC') # 라벨인코딩 처리한 OC열 제외 
not_dummy_col = set(train.columns) - set(dummy_col) # 연속형변수 + OC열

In [17]:
# 원핫인코딩, 더미변수 만들기
temp2 = data[not_dummy_col].copy() # 연속형변수 데이터프레임
for col in dummy_col: 
    ohe = OneHotEncoder(sparse=False)
    temp1 = pd.DataFrame(ohe.fit_transform(data[col].values.reshape(-1,1)), columns = ohe.categories_[0])
    temp2 = pd.concat([temp2, temp1], axis=1)
temp2['OC'] = data['OC'].copy()
data = temp2.copy()

In [18]:
# 기존의 train, test 나누기
train = data.loc[data['inst_id'].isin(train['inst_id'])]
test = data.loc[data['inst_id'].isin(test['inst_id'])]

#### 3. 모델링

In [19]:
train_save = train.copy()
test_save = test.copy()
train = train_save.copy()
test = test_save.copy()

In [20]:
np.random.seed(0)
random_state = 0 # seed
ms = MinMaxScaler()
ss = StandardScaler()
rs = RobustScaler()

train_y = train.pop('OC')
train_x = train.copy()
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.3, stratify=train_y,random_state=random_state)

print(train_y.value_counts())
print(valid_y.value_counts())

1    200
0     10
Name: OC, dtype: int64
1    86
0     5
Name: OC, dtype: int64


In [21]:
train_x = rs.fit_transform(train_x)
valid_x = rs.transform(valid_x)
print(train_x.shape, valid_x.shape)

(210, 82) (91, 82)


In [22]:
# Gradient Boosting
gbm = GradientBoostingClassifier()
gbm.fit(train_x, train_y)
gbm.predict(valid_x)
gbm_pred = gbm.predict_proba(valid_x)

In [23]:
# RandomForest
estimators = 100
rf = RandomForestClassifier(n_estimators = estimators,)
rf.fit(train_x, train_y)
rf.predict(valid_x)
rf_pred = rf.predict_proba(valid_x)

In [24]:
# Kneighbors
knn = KNeighborsClassifier()
knn.fit(train_x, train_y)
knn.predict(valid_x)
knn_pred = knn.predict_proba(valid_x)

In [25]:
# MLP
mlp = MLPClassifier(max_iter=10000)
mlp.fit(train_x, train_y)
mlp.predict(valid_x)
mlp_pred = mlp.predict_proba(valid_x)

In [26]:
# Xgb
xgbm = XGBClassifier()
xgbm.fit(train_x, train_y)
xgbm.predict(valid_x)
xgbm_pred = xgbm.predict_proba(valid_x)

In [27]:
# Lgbm
lgbm = LGBMClassifier()
lgbm.fit(train_x, train_y)
lgbm.predict(valid_x)
lgbm_pred = lgbm.predict_proba(valid_x)

In [28]:
# Cat
cat = CatBoostClassifier()
cat.fit(train_x, train_y)
cat.predict(valid_x)
cat_pred = cat.predict_proba(valid_x)

Learning rate set to 0.005291
0:	learn: 0.6870116	total: 56.5ms	remaining: 56.5s
1:	learn: 0.6798848	total: 64.3ms	remaining: 32.1s
2:	learn: 0.6724983	total: 71.9ms	remaining: 23.9s
3:	learn: 0.6662740	total: 79.8ms	remaining: 19.9s
4:	learn: 0.6596351	total: 87.3ms	remaining: 17.4s
5:	learn: 0.6534650	total: 94.9ms	remaining: 15.7s
6:	learn: 0.6454769	total: 102ms	remaining: 14.5s
7:	learn: 0.6391557	total: 110ms	remaining: 13.7s
8:	learn: 0.6326025	total: 118ms	remaining: 12.9s
9:	learn: 0.6267253	total: 125ms	remaining: 12.4s
10:	learn: 0.6201870	total: 130ms	remaining: 11.7s
11:	learn: 0.6135077	total: 137ms	remaining: 11.3s
12:	learn: 0.6072953	total: 145ms	remaining: 11s
13:	learn: 0.6015688	total: 152ms	remaining: 10.7s
14:	learn: 0.5955138	total: 160ms	remaining: 10.5s
15:	learn: 0.5902209	total: 168ms	remaining: 10.3s
16:	learn: 0.5842475	total: 175ms	remaining: 10.1s
17:	learn: 0.5786058	total: 182ms	remaining: 9.95s
18:	learn: 0.5733134	total: 190ms	remaining: 9.81s
19:	lea

In [29]:
models = [gbm, rf, knn, mlp, xgbm, lgbm, cat]
models_name = ['gbm','rf','knn','mlp','xgbm','lgbm','cat']

In [30]:
# check by model 
for model, name in zip(models, models_name):
    print('##########',name,'##############################')
    print(f'정밀도: {metrics.precision_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'재현율: {recall_score(valid_y, model.predict(valid_x),pos_label=1) * 100:.4f}%')
    print(f'f1_score: {f1_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'정확도: {accuracy_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'ROC_AUC: {roc_auc_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
# gbm 모델이 우수

########## gbm ##############################
정밀도: 95.3488%
재현율: 95.3488%
f1_score: 95.3488%
정확도: 91.2088%
ROC_AUC: 57.6744%
########## rf ##############################
정밀도: 94.3820%
재현율: 97.6744%
f1_score: 96.0000%
정확도: 92.3077%
ROC_AUC: 48.8372%
########## knn ##############################
정밀도: 94.5055%
재현율: 100.0000%
f1_score: 97.1751%
정확도: 94.5055%
ROC_AUC: 50.0000%
########## mlp ##############################
정밀도: 94.3820%
재현율: 97.6744%
f1_score: 96.0000%
정확도: 92.3077%
ROC_AUC: 48.8372%
########## xgbm ##############################
정밀도: 94.3820%
재현율: 97.6744%
f1_score: 96.0000%
정확도: 92.3077%
ROC_AUC: 48.8372%
########## lgbm ##############################
정밀도: 94.3820%
재현율: 97.6744%
f1_score: 96.0000%
정확도: 92.3077%
ROC_AUC: 48.8372%
########## cat ##############################
정밀도: 94.3820%
재현율: 97.6744%
f1_score: 96.0000%
정확도: 92.3077%
ROC_AUC: 48.8372%


In [31]:
# soft voting
cols = list()
for n, m in enumerate(models):
    if n == 0:
        temp1 = pd.DataFrame(m.predict_proba(valid_x)[:,1])
    else:
        temp2 = pd.DataFrame(m.predict_proba(valid_x)[:,1])
        temp1 = pd.concat([temp1, temp2], axis=1)
ensemble = pd.DataFrame(temp1)
ensemble.columns = ['gbm','rf','knn','mlp','xgb','lgbm','cat']
ensemble  = ensemble[['gbm','knn']] ## 보팅모델선택
ensemble['Prob'] = ensemble.sum(axis=1)/len(ensemble.columns)
threshold = 0.7 # check 
ensemble['OC'] = 0
ensemble.loc[ensemble['Prob']>threshold,'OC'] = 1

valid_pred_y = np.array(ensemble['OC']).copy()
print(f'정밀도: {metrics.precision_score(valid_y, valid_pred_y) * 100:.4f}%')
print(f'재현율: {recall_score(valid_y, valid_pred_y,pos_label=1) * 100:.4f}%')
print(f'f1_score: {f1_score(valid_y, valid_pred_y) * 100:.4f}%')
print(f'정확도: {accuracy_score(valid_y, valid_pred_y) * 100:.4f}%')
print(f'ROC_AUC: {roc_auc_score(valid_y, valid_pred_y) * 100:.4f}%')
# gbm + knn -> Trheshold를 조절하며 성능향상 가능성

정밀도: 95.3488%
재현율: 95.3488%
f1_score: 95.3488%
정확도: 91.2088%
ROC_AUC: 57.6744%


#### 하이퍼파라미터튜닝

Optuna를 이용한 하이퍼파라미터  튜닝

In [32]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

def objective(trial: Trial) -> float:
    global train_x, train_y, valid_x, valid_y
    params_lgb = {
        "random_state": 0,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "objective": "binary",
        "metric": "binary_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    model = LGBMClassifier(**params_lgb)
    model.fit(train_x,train_y,
        eval_set=[(train_x, train_y), (valid_x, valid_y)],
        early_stopping_rounds=100,
        verbose=True,
    )
    lgb_pred = model.predict_proba(valid_x)
    log_score = log_loss(valid_y, lgb_pred)
    return log_score

In [33]:
sampler = TPESampler(seed=0)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=100, )
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2023-01-29 15:00:33,063] A new study created in memory with name: lgbm_parameter_opt
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-01-29 15:00:33,267] Trial 0 finished with value: 0.21339083854053043 and parameters: {'reg_alpha': 1.646891698278047e-05, 'reg_lambda': 0.06436704582162409, 'max_depth': 13, 'num_leaves': 140, 'colsample_bytr

[1]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[2]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[3]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[4]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[5]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[6]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[7]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[8]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[9]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[10]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[11]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[12]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[13]	training's binary_logloss: 0.287805	valid_1's binary_logloss: 0.331698
[14]	training's binar

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's binary_logloss: 0.187463	valid_1's binary_logloss: 0.212175
[2]	training's binary_logloss: 0.179658	valid_1's binary_logloss: 0.215306
[3]	training's binary_logloss: 0.171477	valid_1's binary_logloss: 0.216514
[4]	training's binary_logloss: 0.163817	valid_1's binary_logloss: 0.218514
[5]	training's binary_logloss: 0.157756	valid_1's binary_logloss: 0.220303
[6]	training's binary_logloss: 0.157821	valid_1's binary_logloss: 0.221395
[7]	training's binary_logloss: 0.154754	valid_1's binary_logloss: 0.220926
[8]	training's binary_logloss: 0.149976	valid_1's binary_logloss: 0.223766
[9]	training's binary_logloss: 0.143073	valid_1's binary_logloss: 0.225794
[10]	training's binary_logloss: 0.138589	valid_1's binary_logloss: 0.222453
[11]	training's binary_logloss: 0.134906	valid_1's binary_logloss: 0.223427
[12]	training's binary_logloss: 0.133654	valid_1's binary_logloss: 0.224604
[13]	training's binary_logloss: 0.132573	valid_1's binary_logloss: 0.221029
[14]	training's binar

[I 2023-01-29 15:00:33,570] Trial 1 finished with value: 0.21217456238873358 and parameters: {'reg_alpha': 1.1509411149585074e-05, 'reg_lambda': 0.07125525551018942, 'max_depth': 11, 'num_leaves': 146, 'colsample_bytree': 0.9553579829755966, 'subsample': 0.34972524073852085, 'subsample_freq': 1, 'min_child_samples': 6, 'max_bin': 450}. Best is trial 1 with value: 0.21217456238873358.
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verb

...

[1]	training's binary_logloss: 0.190193	valid_1's binary_logloss: 0.211619
[2]	training's binary_logloss: 0.18896	valid_1's binary_logloss: 0.211309
[3]	training's binary_logloss: 0.18811	valid_1's binary_logloss: 0.210164
[4]	training's binary_logloss: 0.186983	valid_1's binary_logloss: 0.208572
[5]	training's binary_logloss: 0.185968	valid_1's binary_logloss: 0.207126
[6]	training's binary_logloss: 0.184804	valid_1's binary_logloss: 0.206896
[7]	training's binary_logloss: 0.183759	valid_1's binary_logloss: 0.206742
[8]	training's binary_logloss: 0.182817	valid_1's binary_logloss: 0.206653
[9]	training's binary_logloss: 0.181863	valid_1's binary_logloss: 0.205321
[10]	training's binary_logloss: 0.180708	valid_1's binary_logloss: 0.203697
[11]	training's binary_logloss: 0.179529	valid_1's binary_logloss: 0.20213
[12]	training's binary_logloss: 0.178345	valid_1's binary_logloss: 0.200514
[13]	training's binary_logloss: 0.177164	valid_1's binary_logloss: 0.198909
[14]	training's binary_l

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-01-29 15:00:57,537] Trial 98 finished with value: 0.19603768299452087 and parameters: {'reg_alpha': 1.6553128877915703e-05, 'reg_lambda': 0.06604888836336652, 'max_depth': 19, 'num_leaves': 194, 'colsample_bytree': 0.7306637546233548, 'subsample': 0.7584687605971062, 'subsample_freq': 9, 'min_ch

[1]	training's binary_logloss: 0.190171	valid_1's binary_logloss: 0.212255
[2]	training's binary_logloss: 0.18905	valid_1's binary_logloss: 0.211278
[3]	training's binary_logloss: 0.188464	valid_1's binary_logloss: 0.212034
[4]	training's binary_logloss: 0.187559	valid_1's binary_logloss: 0.212624
[5]	training's binary_logloss: 0.186758	valid_1's binary_logloss: 0.213255
[6]	training's binary_logloss: 0.185677	valid_1's binary_logloss: 0.213193
[7]	training's binary_logloss: 0.184722	valid_1's binary_logloss: 0.213215
[8]	training's binary_logloss: 0.183875	valid_1's binary_logloss: 0.213309
[9]	training's binary_logloss: 0.182817	valid_1's binary_logloss: 0.211922
[10]	training's binary_logloss: 0.181311	valid_1's binary_logloss: 0.209908
[11]	training's binary_logloss: 0.179785	valid_1's binary_logloss: 0.207856
[12]	training's binary_logloss: 0.178366	valid_1's binary_logloss: 0.205918
[13]	training's binary_logloss: 0.17713	valid_1's binary_logloss: 0.204218
[14]	training's binary_

[I 2023-01-29 15:00:57,754] Trial 99 finished with value: 0.19705611897457162 and parameters: {'reg_alpha': 1.9238270807946642e-05, 'reg_lambda': 0.07817740024207917, 'max_depth': 20, 'num_leaves': 14, 'colsample_bytree': 0.8279262672028098, 'subsample': 0.8202526816641107, 'subsample_freq': 8, 'min_child_samples': 67, 'max_bin': 258}. Best is trial 61 with value: 0.1880523458742202.


[72]	training's binary_logloss: 0.130075	valid_1's binary_logloss: 0.227889
[73]	training's binary_logloss: 0.129145	valid_1's binary_logloss: 0.228375
[74]	training's binary_logloss: 0.128271	valid_1's binary_logloss: 0.228488
[75]	training's binary_logloss: 0.127778	valid_1's binary_logloss: 0.228848
[76]	training's binary_logloss: 0.127366	valid_1's binary_logloss: 0.22926
[77]	training's binary_logloss: 0.126647	valid_1's binary_logloss: 0.229886
[78]	training's binary_logloss: 0.126366	valid_1's binary_logloss: 0.231584
[79]	training's binary_logloss: 0.126046	valid_1's binary_logloss: 0.232051
[80]	training's binary_logloss: 0.125423	valid_1's binary_logloss: 0.23302
[81]	training's binary_logloss: 0.124869	valid_1's binary_logloss: 0.233695
[82]	training's binary_logloss: 0.124386	valid_1's binary_logloss: 0.233857
[83]	training's binary_logloss: 0.123911	valid_1's binary_logloss: 0.234523
[84]	training's binary_logloss: 0.123462	valid_1's binary_logloss: 0.235356
[85]	training'

In [34]:
# Lgbm
tuned_lgbm = LGBMClassifier(**study.best_trial.params)
tuned_lgbm.fit(train_x, train_y)
tuned_lgbm.predict(valid_x)
lgbm_pred = tuned_lgbm.predict_proba(valid_x)

In [35]:
for model, name in zip([lgbm, tuned_lgbm], ['lgbm','tuned_lgbm']):
    print('##########',name,'##########')
    print(f'정밀도: {metrics.precision_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'재현율: {recall_score(valid_y, model.predict(valid_x),pos_label=1) * 100:.4f}%')
    print(f'f1_score: {f1_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'정확도: {accuracy_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'ROC_AUC: {roc_auc_score(valid_y, model.predict(valid_x)) * 100:.4f}%')

########## lgbm ##########
정밀도: 94.3820%
재현율: 97.6744%
f1_score: 96.0000%
정확도: 92.3077%
ROC_AUC: 48.8372%
########## tuned_lgbm ##########
정밀도: 94.5055%
재현율: 100.0000%
f1_score: 97.1751%
정확도: 94.5055%
ROC_AUC: 50.0000%


automl을 통한 튜닝, # scikit-learn-intelex > 21.02, recommend using colab

In [36]:
train = train_save.copy()
test = test_save.copy()
# train['OC'] = train['OC'].apply(lambda x : str(x).replace('0','close'))
# train['OC'] = train['OC'].apply(lambda x : str(x).replace('1','open'))
# test['OC'] = test['OC'].apply(lambda x : str(x).replace('0','close'))
# test['OC'] = test['OC'].apply(lambda x : str(x).replace('1','open'))

In [37]:
from pycaret.classification import *
exp_clf101 = setup(data = train, 
                   target='OC', # target
                   session_id=0, # random_seed
                   normalize=True,
                   normalize_method='robust', # robust scaler
                   fold_strategy='stratifiedkfold', # stratified K-fold Cross Validation
                   ) # already preprocess the label encoding, one-hot encoding 

,Description,Value
0,session_id,0
1,Target,OC
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(301, 83)"
5,Missing Values,False
6,Numeric Features,57
7,Categorical Features,25
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='OC',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy=...
                                                  target='OC')),
                ('P_transform', 'passthrough'), ('binn', 'passthrough'),
                ('rem_outliers', 'pa

In [38]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9810,0.8225,1.000,0.9810,0.9902,0.6000,0.6000,0.206
dt,Decision Tree Classifier,0.9714,0.7950,0.990,0.9810,0.9851,0.5288,0.5378,0.015
ada,Ada Boost Classifier,0.9619,0.7250,0.995,0.9667,0.9804,0.2644,0.2689,0.114
rf,Random Forest Classifier,0.9571,0.8700,1.000,0.9571,0.9780,0.1000,0.1000,0.194
qda,Quadratic Discriminant Analysis,0.9571,0.5975,0.995,0.9617,0.9779,0.1950,0.1950,0.014
et,Extra Trees Classifier,0.9571,0.8325,1.000,0.9571,0.9780,0.1000,0.1000,0.159
lr,Logistic Regression,0.9524,0.6500,0.995,0.9569,0.9755,0.0950,0.0950,0.336
knn,K Neighbors Classifier,0.9524,0.4850,1.000,0.9524,0.9756,0.0000,0.0000,0.019
lightgbm,Light Gradient Boosting Machine,0.9524,0.7500,0.995,0.9569,0.9755,0.0950,0.0950,0.121
catboost,CatBoost Classifier,0.9524,0.8000,1.000,0.9524,0.9756,0.0000,0.0000,9.208


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 2
INFO:logs:GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


In [39]:
final_model = finalize_model(best_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), fit_kwargs=None, groups=None, model_only=True, display=None, experiment_custom_tags=None, return_train_score=False)
INFO:logs:Finalizing GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
              

In [40]:
final_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [41]:
test = test_save.copy()
test_y = test.pop('OC')
test_x = test.copy()
prediction = predict_model(final_model, data = test_x)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading l

In [42]:
# public: 0.9365079365, private: 0.875
score= list() # automl best model
threshold = 0.9
for i in np.array(prediction['Score']):
    if i > threshold:
        score.append(1) # open
    else:
        score.append(0) # close
display(np.array(score))

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1])

In [43]:
# Gradient - automl hyperparmeters
gbm = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

gbm.fit(train_x, train_y)
gbm.predict(valid_x)
gbm_pred = gbm.predict_proba(valid_x)

In [44]:
# public: 0.9523809524, private: 0.921875
test = test_save.copy()
test_y = test.pop('OC')
test_x = test.copy()
test_x = rs.transform(test_x)
score = list()
threshold = 0.94 
for prob in gbm.predict_proba(test_x)[:,1]:
  if prob > threshold:
    score.append(1)
  else:
    score.append(0)
np.array(score)

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])